In [8]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
#scikit-learn related imports
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# pytorch relates imports
import torch
import torch.nn as nn
import torch.optim as optim
import openpyxl

# imports from captum library
from captum.attr import IntegratedGradients,  GradientShap,FeatureAblation
from sklearn.utils import resample
from sklearn.datasets import fetch_california_housing
california = fetch_california_housing()

import scipy.stats as stats

BD_path = '../data/reducedata_bd1011.xlsx'
HC_path = '../data/reducedata_hc1011.xlsx'
HC = pd.read_excel(HC_path, index_col = False)
HC = HC.iloc[:, 2:]
HC.columns = HC.iloc[0]
HC = HC.iloc[1:].reset_index(drop=True)

BD = pd.read_excel(BD_path, index_col = False)
BD = BD.iloc[:, 2:]
BD.columns = BD.iloc[0]
BD = BD.iloc[1:].reset_index(drop=True)
BD_mean = BD.iloc[:, 2:].mean()
BD_sd = BD.iloc[:, 2:].std()
HC_mean = HC.iloc[:, 2:].mean()
HC_sd = HC.iloc[:, 2:].std()

HC_synthetic = np.random.normal(loc=HC_mean.T, scale=HC_sd.T, size=(120, 88))
HC_synthetic = pd.DataFrame(HC_synthetic)
HC_synthetic.columns = HC_mean.index

BD_synthetic = np.random.normal(loc=BD_mean.T, scale=BD_sd.T, size=(60, 88))
BD_synthetic = pd.DataFrame(BD_synthetic)
BD_synthetic.columns = BD_mean.index

In [9]:
BD_synthetic.to_csv('../data/BD_synthetic.csv')
HC_synthetic.to_csv('../data/HC_synthetic.csv')

In [14]:
#stats.ttest_ind(BD_synthetic, HC_synthetic)